#Fine-Tuning Whisper Small Checkpoint For Serbian Language

Checking if GPU is properly allocated for us:

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a8637e0d-313f-69eb-7e0e-e2b16d493833)


Installing requirenments:

In [2]:
!pip install datasets>=2.6.1
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 22.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [3]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hrnraw9n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hrnraw9n
  Resolved https://github.com/huggingface/transformers to commit e16191a8ac118cea95b3353fc837e288edae64be
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7283130 sha256=9344e2f0d6dc12cab859e5d215c428df8ebb225c05215bc154a4dad3f252a83f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ymy9mefr/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


Logging to HuggingFace hub to gain access to models, datasets and be able to push our training model and checkpoints to their hub for backup and sharing:

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Loading the dataset (*mozilla-foundation/common_voice_11_0*). The train and validation split are combined to make up a larger dataset for training the model (after the main parameters were determined by training on the solo train split), why the test split was used for the evaluation portion of the training.

The combined training split thus ammounts to around 1.4h of audio (1600+ audio recordings with average length of 4s), while the test split contains a bit less than 700 recordings with simular average length.

In [5]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

# common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sr", split="train", use_auth_token=True)
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sr", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sr", split="test", use_auth_token=True)

print(common_voice["train"]["sentence"])

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1045it [00:00, 67687.67it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 61it [00:00, 76373.89it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 42it [00:00, 66955.82it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/sr/11.0.0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631. Subsequent calls will reuse this data.


['То све доказује.', 'А не може.', 'Он је пријатељ.', 'То не може?', 'Као и пре?', 'Гуча је дивна.', 'И о томе.', 'Што да нисам?', 'Е, не видим.', 'Мир и тишину.', 'Не знам шта?', 'Чета је резултат многих компромиса.', 'А сви ови?', 'Од слонова је најобичнији био јужни слон', 'Шта то ради!', 'Не, то је природно.', 'Ето ти сад!', 'И моја је.', 'Да се забавља.', 'Данас је то објекат за понос.', 'Хвала вам велика.', 'И још како.', 'То је био?', 'И никад није.', 'Ја сам то.', 'О то си ти?', 'Молим те, немој!', 'И где је?', 'Где је, где?', 'У разне животиње.', 'За све то?', 'Шта је био проблем?', 'Било је време за акцију.', 'Супер смо се провели.', 'Могу ја то.', 'Добар дан, господине!', 'То је у овоме?', 'То није тако.', 'Како се ради?', 'И ја бих.', 'Ситуацију би најбоље могао да сумира обичан грађанин.', 'И због тога?', 'То, међутим, треба да буду професионалне снаге.', 'Какав је то?', 'Па, била је!', 'А не можеш.', 'Ја мислим овако.', 'А, за шта?', 'А све ово?', 'Шта ћеш сад?', 'А треба

Removing unnecessary columns from the datasets (we only need "audio" and "sentence"):

In [6]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

The dataset uses cyrillic script, to convert it to latin we're going to be using the *transliterate* library - installing and converting the sentences of all dataset splits:

In [7]:
!pip install transliterate
from transliterate import translit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.8 MB/s eta 0:00:00


In [8]:
import numpy as np

transliteration_function = np.vectorize(translit)

def transliterate_text(example):
    trans = transliteration_function(example["sentence"], 'sr', reversed=True)
    example["sentence"] = trans
    # print(trans)
    return example

common_voice["train"] = common_voice["train"].map(transliterate_text)
common_voice["test"] = common_voice["test"].map(transliterate_text)

common_voice = common_voice.map(lambda x: {"sentence": [sent for sublist in x["sentence"] for sent in sublist]}, batched=True)

# print(common_voice["train"]["sentence"])
# print(common_voice["test"]["sentence"])




Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

Map:   0%|          | 0/677 [00:00<?, ? examples/s]

Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

Map:   0%|          | 0/677 [00:00<?, ? examples/s]

In [9]:
print(common_voice["train"]["sentence"])
print(common_voice["test"]["sentence"])

['To sve dokazuje.', 'A ne može.', 'On je prijatelj.', 'To ne može?', 'Kao i pre?', 'Guča je divna.', 'I o tome.', 'Što da nisam?', 'E, ne vidim.', 'Mir i tišinu.', 'Ne znam šta?', 'Četa je rezultat mnogih kompromisa.', 'A svi ovi?', 'Od slonova je najobičniji bio južni slon', 'Šta to radi!', 'Ne, to je prirodno.', 'Eto ti sad!', 'I moja je.', 'Da se zabavlja.', 'Danas je to objekat za ponos.', 'Hvala vam velika.', 'I još kako.', 'To je bio?', 'I nikad nije.', 'Ja sam to.', 'O to si ti?', 'Molim te, nemoj!', 'I gde je?', 'Gde je, gde?', 'U razne životinje.', 'Za sve to?', 'Šta je bio problem?', 'Bilo je vreme za akciju.', 'Super smo se proveli.', 'Mogu ja to.', 'Dobar dan, gospodine!', 'To je u ovome?', 'To nije tako.', 'Kako se radi?', 'I ja bih.', 'Situaciju bi najbolje mogao da sumira običan građanin.', 'I zbog toga?', 'To, međutim, treba da budu profesionalne snage.', 'Kakav je to?', 'Pa, bila je!', 'A ne možeš.', 'Ja mislim ovako.', 'A, za šta?', 'A sve ovo?', 'Šta ćeš sad?', 'A t

Setting locale to UTF-8 for later printing the sentences in the correct encoding:

In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.3 MB/s eta 0:00:00


Data augmentation step - to further diversify the "small" training set, we're applying simple audio augmantation functions to the train split - an effort to try to get better model generalization:

In [11]:

from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

In [12]:
augment_waveform = Compose([
    AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=0.2),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.2, leave_length_unchanged=False),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.2)
    ,])

def augment_dataset(batch):

    audio = batch["audio"]["array"]
    # apply augmentation
    augmented_audio = augment_waveform(samples=audio, sample_rate=16000)

    batch["audio"]["array"] = augmented_audio

    return batch


# call augment dataset on the training set
common_voice["train"] = common_voice["train"].map(augment_dataset)

Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Loading the Whisper Small pretrained checkpoint and all its key components:

In [13]:
from transformers import WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Serbian", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Serbian", task="transcribe")

For dataset preparation step, we're using the default prepare_dataset() function provided in the [official Fine-Tuning event example](https://huggingface.co/blog/fine-tune-whisper).

Key points for data preperation:


*   Convert all audio to 16000Hz sampling rate
*   Compute log-Mel spectograms from the audio using feature-extractor
*   Encode target sentences (transcriptions) using the tokenizer



In [14]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=16000).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=3).with_format("torch")




Map (num_proc=3):   0%|          | 0/1668 [00:00<?, ? examples/s]

Map (num_proc=3):   0%|          | 0/677 [00:00<?, ? examples/s]

Shuffling the dataset for added randomness:

In [15]:
common_voice["train"] = common_voice["train"].shuffle(

)

Using the data collator provided in the [official Fine-Tuning event example](https://huggingface.co/blog/fine-tune-whisper).

The data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

In [16]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Mounting our GoogleDrive account for added backup and unexpected "Runtime Disconected" situation handling (highly suggested).

We'll later utilize our Drive in the compute_metrics() function to save all checkpoints for our model, enabling us to resume/load checkpoints even if the runtime data is lost.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Set the output directory where the checkpoints will be saved
output_dir = '/content/drive/MyDrive/checkpoints/'

Mounted at /content/drive


In [18]:
import datetime

def save_checkpoint(trainer, model, output_dir):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    trainer.save_model(output_dir + f"/checkpoint_{timestamp}")

In [2]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
print(timestamp)

20230703_192938


Compute metrics function from the original Fine-Tuning blog post, upgraded with:
* logging the predictions and references to the *refs_and_preds.txt* file - this could be very useful for analyzing the model output and tweaking the parameters based on the conclusions.
* saving checkpoint to GoogleDrive

In [19]:
import evaluate

metric = evaluate.load("wer")

# evaluate with the 'normalised' WER
do_normalize_eval = True

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True, normalize=do_normalize_eval)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True, normalize=do_normalize_eval)

    with open('refs_and_preds.txt', 'w') as f:
      for ref, pred in zip(label_str, pred_str):
          f.write(f"Ref: {ref.encode('utf-8')}\n")
          f.write(f"Pred: {pred.encode('utf-8')}\n\n")

    save_checkpoint(trainer, model, output_dir)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [20]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []



By analyzing the refs_and_preds.txt file, it was noticed that the model transcribed to a wrong language, or script, or both, or just gave out pure empty strings as predictions not too rarely - this impacted WER very badly, and this simple line of code telling the model which language to use improved upon these issues pretty nicely:

In [21]:
from functools import partial
model.generate = partial(model.generate, language="Serbian", task="transcribe")

For analitycs and nice logging, wandb library was used:

In [22]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=43b64efdf5a7957f7b51ad698d41281076de002fb0188d617077aa6957046bac
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [24]:
%set_env WANDB_LOG_MODEL=True
%set_env WANDB_WATCH=all
%set_env WANDB_NOTEBOOK_NAME=whisper_small_sr_rs_ri_finale

%env WANDB_PROJECT=whisper_small_sr_rs_ri_finale_proj

import wandb
wandb.login()
# run = wandb.init(project="whisper_finetuning_sr", job_type="fine-tuning", group="small" )

env: WANDB_LOG_MODEL=True
env: WANDB_WATCH=all
env: WANDB_NOTEBOOK_NAME=whisper_small_sr_rs_ri_finale
env: WANDB_PROJECT=whisper_small_sr_rs_ri_finale_proj


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Finally, setting the training arguments and initializing the trainer. This is the current best configuration:

In [25]:
from torch.nn.modules import dropout
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

model.config.dropout = 0.2

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-sr-ri",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=250,
    eval_steps=250,
    logging_steps=25,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    dataloader_num_workers=2,
)



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

processor.save_pretrained(training_args.output_dir)

Cloning https://huggingface.co/tince250/whisper-small-sr-ri into local empty directory.


Download file pytorch_model.bin:   0%|          | 32.0k/922M [00:00<?, ?B/s]

Download file runs/Jul01_18-53-07_9d4032991f1d/events.out.tfevents.1688237597.9d4032991f1d.5756.8: 100%|######…

Download file runs/Jul02_08-31-30_5e002a51d4d2/events.out.tfevents.1688286700.5e002a51d4d2.344.2: 100%|#######…

Download file runs/Jul01_17-11-55_9d4032991f1d/events.out.tfevents.1688231525.9d4032991f1d.5756.7: 100%|######…

...

Clean file runs/Jul02_11-17-35_5e002a51d4d2/events.out.tfevents.1688296681.5e002a51d4d2.344.3:  21%|##        …

Clean file runs/Jul02_08-29-24_5e002a51d4d2/events.out.tfevents.1688286575.5e002a51d4d2.344.0:  21%|##        …

Download file training_args.bin: 100%|##########| 4.00k/4.00k [00:00<?, ?B/s]

Clean file training_args.bin:  25%|##5       | 1.00k/4.00k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/922M [00:00<?, ?B/s]

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [27]:
trainer.train(resume_from_checkpoint="./whisper-small-sr-ri/checkpoint-1500")

Step,Training Loss,Validation Loss,Wer
1750,0.000300,0.535277,19.174757
2000,0.000300,0.537420,19.174757


Step,Training Loss,Validation Loss,Wer
250,0.095400,0.326219,19.862460
500,0.033200,0.363609,20.711974
750,0.005000,0.455462,19.053398
1000,0.001400,0.513732,18.891586
1250,0.000500,0.515995,18.810680
1500,0.000300,0.527528,19.215210


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/922M [00:00<?, ?B/s]

To https://huggingface.co/tince250/whisper-small-sr-ri
   997d088..030a307  main -> main

   997d088..030a307  main -> main

To https://huggingface.co/tince250/whisper-small-sr-ri
   030a307..de911ef  main -> main

   030a307..de911ef  main -> main



Upload file pytorch_model.bin:   0%|          | 1.00/922M [00:00<?, ?B/s]

To https://huggingface.co/tince250/whisper-small-sr-ri
   de911ef..1a5edb9  main -> main

   de911ef..1a5edb9  main -> main

To https://huggingface.co/tince250/whisper-small-sr-ri
   1a5edb9..8638d45  main -> main

   1a5edb9..8638d45  main -> main

/content/./whisper-small-sr-ri is already a clone of https://huggingface.co/tince250/whisper-small-sr-ri. Make sure you pull the latest changes with `repo.git_pull()`.
To https://huggingface.co/tince250/whisper-small-sr-ri
   8638d45..7ab4c1a  main -> main

   8638d45..7ab4c1a  main -> main

To https://huggingface.co/tince250/whisper-small-sr-ri
   7ab4c1a..ea835a1  main -> main

   7ab4c1a..ea835a1  main -> main



TrainOutput(global_step=2000, training_loss=7.768395659513772e-05, metrics={'train_runtime': 3134.4602, 'train_samples_per_second': 10.209, 'train_steps_per_second': 0.638, 'total_flos': 9.16893533896704e+18, 'train_loss': 7.768395659513772e-05, 'epoch': 19.05})

In [ ]:
model.load_state_dict(torch.load(path))

To showcase and test the models performance in a minimalistic, generic Web App UI demo, *gradio* library is utilized:

In [28]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="tince250/whisper-small-sr-ri")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Serbian",
    description="Realtime demo for Serbian speech recognition using a fine-tuned Whisper small model.",
)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [29]:
iface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2a46f592d9b694b1ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
